In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## LOADING

In [3]:
basedir = '/Users/sahara/Documents/GW/ML1/nhlgames/'

In [322]:
game = pd.read_csv(f'{basedir}/game_data/game.csv')
team_stats = pd.read_csv(f'{basedir}/game_data/game_teams_stats.csv')
team_info = pd.read_csv(f'{basedir}/game_data/team_info.csv')

In [323]:
game = game[game['type'].isin(['R','P'])]
team_stats = team_stats[team_stats['settled_in'].isin(['REG', 'OT'])]
datgame = game[['game_id', 'season', 'date_time_GMT']]
datteam = team_stats[['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach', 'shots', 'hits', 'pim', 'powerPlayOpportunities', 'faceOffWinPercentage', 'giveaways', 'takeaways', 'blocked']]

dat = datteam.merge(datgame, on = 'game_id', how = 'inner')
dat = dat.drop_duplicates(subset = ['game_id', 'team_id'])

dat = dat.sort_values(by = ['team_id', 'date_time_GMT'])

In [324]:
print(dat.shape)
print(datteam.shape)
print(datgame.shape)

(47432, 16)
(52562, 14)
(26295, 3)


In [326]:
dat.index.unique()

Int64Index([37411, 36124, 38299,  8804, 36870, 37007, 37078, 36844,  7563,
            38313,
            ...
            51710, 51734, 51768, 51790, 51808, 51824, 51840, 51854, 51870,
            51888],
           dtype='int64', length=47432)

## Feature Engineering

In [327]:
dat['pts'] = dat.apply(lambda x: 2 if x.won else 1 if not x.won and x.settled_in == 'OT' else 0, axis = 1)

In [328]:
dat.index.unique()

Int64Index([37411, 36124, 38299,  8804, 36870, 37007, 37078, 36844,  7563,
            38313,
            ...
            51710, 51734, 51768, 51790, 51808, 51824, 51840, 51854, 51870,
            51888],
           dtype='int64', length=47432)

In [329]:
seasonsorted = sorted(dat['season'].unique())
dat['season_num'] = dat['season'].apply(lambda x: seasonsorted.index(x))
grouped = dat.groupby(['team_id', 'season_num']).agg({'game_id':'count', 'pts':'sum'})
grouped = grouped.reset_index()
grouped['pts_perc_last_szn'] = grouped['pts']/(grouped['game_id']*2)
grouped['season_num'] = grouped['season_num']+1

to_merge = grouped[['team_id', 'season_num', 'pts_perc_last_szn']]
dat = dat.merge(to_merge, on = ['season_num', 'team_id'], how = 'left')

In [330]:
dat.index.unique()

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            47422, 47423, 47424, 47425, 47426, 47427, 47428, 47429, 47430,
            47431],
           dtype='int64', length=47432)

In [331]:
dat['L10_pts_perc'] = dat.groupby(['team_id', 'season_num'])['pts'].transform(lambda x: x.rolling(10,10).sum()/20)
dat['rolling_pts%'] = dat.groupby(['team_id', 'season_num']).apply(lambda x: x.pts.expanding().sum()/(x.pts.expanding().count()*2)).values


In [332]:
dat.index.unique()

Int64Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,
                9,
            ...
            47422, 47423, 47424, 47425, 47426, 47427, 47428, 47429, 47430,
            47431],
           dtype='int64', length=47432)

## Transforming

In [333]:
dat['home'] = dat['HoA'].apply(lambda x: 1 if x=='home' else 0)

In [334]:
dat['date_time_GMT'] = pd.to_datetime(dat['date_time_GMT'])

In [335]:
dat['gametime_unix'] = dat['date_time_GMT'].apply(lambda x: x.timestamp())

In [336]:
coach_group = dat.groupby(['head_coach']).apply(lambda x: x.won.expanding().sum()/(x.won.expanding().count()*2))

In [337]:
coach_flat = coach_group.reset_index().rename(columns = {'won':'coach_pts%'})

In [338]:
dat = dat.merge(coach_flat, left_index=True, right_on = 'level_1')

In [339]:
dat.columns

Index(['game_id', 'team_id', 'HoA', 'won', 'settled_in', 'head_coach_x',
       'shots', 'hits', 'pim', 'powerPlayOpportunities',
       'faceOffWinPercentage', 'giveaways', 'takeaways', 'blocked', 'season',
       'date_time_GMT', 'pts', 'season_num', 'pts_perc_last_szn',
       'L10_pts_perc', 'rolling_pts%', 'home', 'gametime_unix', 'head_coach_y',
       'level_1', 'coach_pts%'],
      dtype='object')

In [340]:
df = dat.drop(['HoA', 'head_coach_x', 'date_time_GMT', 'season', 'head_coach_y', 'level_1', 'settled_in'], axis = 1)

In [341]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 47432 entries, 28296 to 37926
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   game_id                 47432 non-null  int64  
 1   team_id                 47432 non-null  int64  
 2   won                     47432 non-null  bool   
 3   shots                   47432 non-null  float64
 4   hits                    42512 non-null  float64
 5   pim                     47432 non-null  float64
 6   powerPlayOpportunities  47432 non-null  float64
 7   faceOffWinPercentage    25292 non-null  float64
 8   giveaways               42512 non-null  float64
 9   takeaways               42512 non-null  float64
 10  blocked                 42512 non-null  float64
 11  pts                     47432 non-null  int64  
 12  season_num              47432 non-null  int64  
 13  pts_perc_last_szn       44706 non-null  float64
 14  L10_pts_perc            42275 non-

In [342]:
dat.index.unique()

Int64Index([28296, 28297, 28298, 28299, 28300, 28301, 28302, 28303, 28304,
            28305,
            ...
            37917, 37918, 37919, 37920, 37921, 37922, 37923, 37924, 37925,
            37926],
           dtype='int64', length=47432)